# Origin and goals of commonlib

1. commonlib was written as a basis for a future lite version of the engine, with no dependency on Postgresql, Django and Celery
2. the idea is to move in commonlib *everything* which does not depend on Postgresql, Django and Celery
3. the engine *must* become small
4. it must be possible to write hazard/risk calculators entirely in commonlib
5. sooner or later the engine QA test will be moved in commonlib
6. hazardlib, risklib, commonlib might become a single repository (oq-lite)

# Use of commonlib: reading the engine input files

In [1]:
import os; print os.getcwd()
print [f for f in os.listdir('.') if f.endswith(('.py', '.ini', '.xml'))]

/home/michele/oq-risklib/doc
['fragility_model.xml', 'job_hc_from_csv.ini', 'job_usa.ini', 'source_model_logic_tree.xml', 'NSHMP2008US_micro.xml', 'fault_rupture.xml', 'hello_test.py', 'exposure_model.xml', 'gmpe_logic_tree.xml', 'expected.xml', 'rupture_model.xml', 'job_gmvs_from_csv.ini', 'job_risk.ini', 'job.ini', 'riskinput.py', 'conf.py', 'hello.ini', 'test.py', 'job_damage.ini', 'vulnerability_model.xml']


In [2]:
print open('job.ini').read()

[general]
description = Scenario QA Test, Case 1
calculation_mode = my_scenario
export_dir = /tmp
random_seed = 3
sites = 0.0 0.0, 0.0 0.1, 0.0 0.2
rupture_mesh_spacing = 1.0
reference_vs30_type = measured
reference_vs30_value = 800.0
reference_depth_to_2pt5km_per_sec = 2.5
reference_depth_to_1pt0km_per_sec = 100.0
rupture_model_file = rupture_model.xml
intensity_measure_types = PGA
truncation_level = 1.0
maximum_distance = 200
gsim = BooreAtkinson2008
number_of_ground_motion_fields = 5



In [4]:
from openquake.baselib.general import import_all; import_all('my_calculators')

set()

In [5]:
from openquake.commonlib import readinput

In [6]:
oqparam = readinput.get_oqparam('job.ini'); oqparam

<OqParam base_path=u'/home/michele/oq-risklib/doc', calculation_mode='my_scenario', complex_fault_mesh_spacing=1.0, description=u'Scenario QA Test, Case 1', export_dir=u'/tmp', gsim='BooreAtkinson2008()', hazard_imtls={'PGA': None}, inputs={'rupture_model': '/home/michele/oq-risklib/doc/rupture_model.xml', 'job_ini': '/home/michele/oq-risklib/doc/job.ini'}, maximum_distance=200.0, number_of_ground_motion_fields=5, random_seed=3, reference_depth_to_1pt0km_per_sec=100.0, reference_depth_to_2pt5km_per_sec=2.5, reference_vs30_type='measured', reference_vs30_value=800.0, risk_investigation_time=None, rupture_mesh_spacing=1.0, sites=[(0.0, 0.0), (0.0, 0.1), (0.0, 0.2)], truncation_level=1.0>

In [7]:
for name, value in oqparam:
    print name, '=', repr(value)

base_path = u'/home/michele/oq-risklib/doc'
calculation_mode = 'my_scenario'
complex_fault_mesh_spacing = 1.0
description = u'Scenario QA Test, Case 1'
export_dir = u'/tmp'
gsim = 'BooreAtkinson2008()'
hazard_imtls = {'PGA': None}
inputs = {'rupture_model': '/home/michele/oq-risklib/doc/rupture_model.xml', 'job_ini': '/home/michele/oq-risklib/doc/job.ini'}
maximum_distance = 200.0
number_of_ground_motion_fields = 5
random_seed = 3
reference_depth_to_1pt0km_per_sec = 100.0
reference_depth_to_2pt5km_per_sec = 2.5
reference_vs30_type = 'measured'
reference_vs30_value = 800.0
risk_investigation_time = None
rupture_mesh_spacing = 1.0
sites = [(0.0, 0.0), (0.0, 0.1), (0.0, 0.2)]
truncation_level = 1.0


In [8]:
for coords in oqparam.sites: print coords

(0.0, 0.0)
(0.0, 0.1)
(0.0, 0.2)


In [9]:
mesh = readinput.get_mesh(oqparam)
for site in mesh: print site

<Latitude=0.000000, Longitude=0.000000, Depth=0.0000>
<Latitude=0.100000, Longitude=0.000000, Depth=0.0000>
<Latitude=0.200000, Longitude=0.000000, Depth=0.0000>


In [27]:
[gsim] = readinput.get_gsims(oqparam); gsim

'BooreAtkinson2008()'

In [12]:
print open('rupture_model.xml').read()

<?xml version='1.0' encoding='utf-8'?>
<nrml xmlns:gml="http://www.opengis.net/gml"
      xmlns="http://openquake.org/xmlns/nrml/0.4">
    <simpleFaultRupture>
        <magnitude>7.0</magnitude>
        <rake>90.0</rake>
        <hypocenter lat="0.0" lon="0.0" depth="10.0"/>
        <simpleFaultGeometry>
            <gml:LineString>
                <gml:posList>
                    28 80.3
                    28  80.6
                </gml:posList>
            </gml:LineString>
            <dip>90.0</dip>
            <upperSeismoDepth>2.0</upperSeismoDepth>
            <lowerSeismoDepth>20.0</lowerSeismoDepth>
        </simpleFaultGeometry>
    </simpleFaultRupture>
</nrml>



In [13]:
rupture = readinput.get_rupture(oqparam); rupture

In [14]:
rupture.mag, rupture.rake, rupture.surface

(7.0,
 90.0,
 <openquake.hazardlib.geo.surface.simple_fault.SimpleFaultSurface at 0x7f0ed6b06050>)

In [15]:
imts = readinput.get_imts(oqparam); imts

[PGA()]

In [16]:
type(imts[0])

openquake.hazardlib.imt.PGA

In [17]:
sitecol = readinput.get_site_collection(oqparam); sitecol

<SiteCollection with 3 sites>

In [18]:
list(sitecol)

[<Location=<Latitude=0.000000, Longitude=0.000000, Depth=0.0000>, Vs30=800.0000, Vs30Measured=True, Depth1.0km=100.0000, Depth2.5km=2.5000, Backarc=False>,
 <Location=<Latitude=0.100000, Longitude=0.000000, Depth=0.0000>, Vs30=800.0000, Vs30Measured=True, Depth1.0km=100.0000, Depth2.5km=2.5000, Backarc=False>,
 <Location=<Latitude=0.200000, Longitude=0.000000, Depth=0.0000>, Vs30=800.0000, Vs30Measured=True, Depth1.0km=100.0000, Depth2.5km=2.5000, Backarc=False>]

## This is enough for performing simple hazard calculations

In [19]:
from openquake.hazardlib.calc.gmf import ground_motion_fields

In [20]:
ground_motion_fields?

In [28]:
ground_motion_fields(rupture, sitecol, imts, gsim, oqparam.truncation_level, 
                     oqparam.number_of_ground_motion_fields,
                     correlation_model=readinput.get_correl_model(oqparam),
                     seed=oqparam.random_seed) 

{PGA(): array([[  1.29778834e-47,   1.12319707e-47,   1.01915100e-47,
           1.07825099e-47,   1.65947553e-47],
        [  2.04784726e-47,   7.51203345e-48,   1.06991019e-47,
           7.86882106e-48,   1.24054768e-47],
        [  1.01174143e-47,   1.16454566e-47,   1.80072257e-47,
           1.13539698e-47,   1.73028861e-47]])}

# Reading risk input files

In [29]:
oqp = readinput.get_oqparam('job_damage.ini')
for name, value in oqp:
    print name, value

base_path /home/michele/oq-risklib/doc
calculation_mode scenario_damage
complex_fault_mesh_spacing 10.0
description Scenario Damage Example
export_dir /tmp
ground_motion_correlation_model None
ground_motion_correlation_params {}
gsim ChiouYoungs2008
inputs {'rupture_model': '/home/michele/oq-risklib/doc/fault_rupture.xml', 'fragility': '/home/michele/oq-risklib/doc/fragility_model.xml', 'job_ini': '/home/michele/oq-risklib/doc/job_damage.ini', 'exposure': '/home/michele/oq-risklib/doc/exposure_model.xml'}
maximum_distance 300.0
number_of_ground_motion_fields 5
random_seed 3
reference_depth_to_1pt0km_per_sec 100.0
reference_depth_to_2pt5km_per_sec 5.0
reference_vs30_type measured
reference_vs30_value 760.0
region_constraint POLYGON((78.0 31.5, 89.5 31.5, 89.5 25.5, 78.0 25.5, 78.0 31.5))
risk_imtls {'SA(0.5)': [0.02, 0.09789473684210527, 0.17578947368421052, 0.25368421052631579, 0.33157894736842108, 0.40947368421052632, 0.48736842105263162, 0.56526315789473691, 0.64315789473684215, 0.72

In [30]:
exposure = readinput.get_exposure(oqp); exposure

Exposure(id='ep', category='buildings', description=u'Exposure model for buildings', cost_types=[{'type': 'per_asset', 'name': 'structural', 'unit': 'USD'}], insurance_limit_is_absolute=True, deductible_is_absolute=True, area={'type': ''}, assets=[<Asset a1>, <Asset a2>, <Asset a3>], taxonomies=set(['RM', 'W', 'RC']))

In [31]:
a1 = exposure.assets[0]

In [32]:
a1.number

3000.0

In [33]:
a1.location

(81.2985, 29.1098)

In [34]:
help(readinput.get_exposure)

Help on function get_exposure in module openquake.commonlib.readinput:

get_exposure(oqparam)
    Read the full exposure in memory and build a list of
    :class:`openquake.risklib.workflows.Asset` instances.
    If you don't want to keep everything in memory, use
    get_exposure_lazy instead (for experts only).
    
    :param oqparam:
        an :class:`openquake.commonlib.oqvalidation.OqParam` instance
    :returns:
        an :class:`Exposure` instance



In [35]:
sitecol, assets_by_site = readinput.get_sitecol_assets(oqp, exposure)
sitecol, assets_by_site

(<SiteCollection with 3 sites>, array([[<Asset a1>],
        [<Asset a2>],
        [<Asset a3>]], dtype=object))

In [36]:
risk_model = readinput.get_risk_model(oqp); risk_model

<RiskModel(3, 0)
('PGA', 'RM'): <Damage['damage']>
('SA(0.2)', 'RC'): <Damage['damage']>
('SA(0.5)', 'W'): <Damage['damage']>>

In [37]:
risk_model.keys()

[('PGA', 'RM'), ('SA(0.2)', 'RC'), ('SA(0.5)', 'W')]

In [38]:
workflow = risk_model['PGA', 'RM']; workflow

<Damage['damage']>

In [39]:
workflow.risk_functions

{'damage': <FragilityFunctionList steps_per_interval=0, continuous_fragility_discretization=20, no_damage_limit=None, imt=PGA, imls=[0.02, 0.09789473684210527, 0.17578947368421052, 0.25368421052631579, 0.33157894736842108, 0.40947368421052632, 0.48736842105263162, 0.56526315789473691, 0.64315789473684215, 0.72105263157894739, 0.79894736842105263, 0.87684210526315798, 0.95473684210526322, 1.0326315789473686, 1.1105263157894738, 1.188421052631579, 1.2663157894736843, 1.3442105263157895, 1.4221052631578948, 1.5]>}

In [40]:
print open('fragility_model.xml').read()

<?xml version='1.0' encoding='utf-8'?>
<nrml xmlns="http://openquake.org/xmlns/nrml/0.4">
  <fragilityModel format="continuous">
    <description>Fragility model for buildings</description>
    <limitStates>
      LS1
      LS2
    </limitStates>
    <ffs type="lognormal">
      <taxonomy>RM</taxonomy>
      <IML IMT="PGA" imlUnit="g" minIML="0.02" maxIML="1.5"/>
      <ffc ls="LS1">
	<params mean="0.15" stddev="0.10" />
      </ffc>
      <ffc ls="LS2">
	<params mean="0.30" stddev="0.16" />
      </ffc>
    </ffs>
    <ffs type="lognormal">
      <taxonomy>RC</taxonomy>
      <IML IMT="SA(0.2)" imlUnit="g" minIML="0.02" maxIML="1.5"/>
      <ffc ls="LS1">
	<params mean="0.25" stddev="0.08" />
      </ffc>
      <ffc ls="LS2">
	<params mean="0.55" stddev="0.16" />
      </ffc>
    </ffs>
    <ffs type="lognormal">
      <taxonomy>W</taxonomy>
      <IML IMT="SA(0.5)" imlUnit="g" minIML="0.02" maxIML="1.5"/>
      <ffc ls="LS1">
	<params mean="0.35" stddev="0.12" />
      </ffc>
      <

In [41]:
workflow.risk_functions['damage'][0], workflow.risk_functions['damage'][1]

(<FragilityFunctionContinuous(LS1, 0.15, 0.1)>,
 <FragilityFunctionContinuous(LS2, 0.3, 0.16)>)

In [42]:
risk_model.damage_states

['no_damage', 'LS1', 'LS2']

In [43]:
?workflow

In [44]:
oqr = readinput.get_oqparam('job_risk.ini')
rm = readinput.get_risk_model(oqr)
rm.keys()

[('PGA', 'RM'), ('SA(0.2)', 'RC'), ('SA(0.5)', 'W')]

In [45]:
rm['SA(0.2)', 'RC'].risk_functions

{'structural': <VulnerabilityFunction(RC, SA(0.2))>}

# Reading sources

In [46]:
from openquake.commonlib import readinput
usparam = readinput.get_oqparam('job_usa.ini')
sitecol = readinput.get_site_collection(usparam)

In [47]:
source_models = list(readinput.get_composite_source_model(usparam, sitecol))  # slow
source_models

[SourceModel(name='NSHMP2008US_micro.xml', weight=Decimal('1.00'), path=('b1',), trt_models=[<TrtModel #0 Active Shallow Crust, 5378 source(s), 9310 rupture(s)>], gsim_lt=<openquake.commonlib.logictree.GsimLogicTree object at 0x7f0ed65b2410>, ordinal=0, samples=1, num_sources=None)]

In [48]:
trt_model = source_models[0].trt_models[0]
trt_model.min_mag, trt_model.max_mag, trt_model.num_ruptures

(5.369, 8.42, 9310)

In [49]:
trt_model.sources[:3], trt_model.sources[-3:]

([<CharacteristicFaultSource 1>,
  <CharacteristicFaultSource 100>,
  <CharacteristicFaultSource 1000>],
 [<CharacteristicFaultSource 997>,
  <CharacteristicFaultSource 998>,
  <CharacteristicFaultSource 999>])

In [50]:
list(trt_model.sources[0].iter_ruptures())

In [51]:
src = trt_model.sources[0]
src.mfd

<EvenlyDiscretizedMFD>

# Reading CSV files

In [52]:
o = readinput.get_oqparam('job_gmvs_from_csv.ini')
readinput.get_sitecol_gmfs(o)

ValueError: It is possible to infer the geometry only if exactly         one of
sites, sites_csv, hazard_curves_csv, gmfs_csv,         region and
exposure_file is set. You did set more than         one, or nothing.
Got:
base_path=/home/michele/oq-risklib/doc
calculation_mode=scenario
description=Example of CSV reading
export_dir=/tmp
hazard_imtls={'PGV': [0.1, 0.2, 0.3], 'PGA': [0.1, 0.2, 0.3]}
inputs={'job_ini': '/home/michele/oq-risklib/doc/job_gmvs_from_csv.ini', 'gmfs': '/home/michele/oq-risklib/doc/gmf.csv'}
maximum_distance=200.0
number_of_ground_motion_fields=3
random_seed=3
reference_depth_to_1pt0km_per_sec=100.0
reference_depth_to_2pt5km_per_sec=2.5
reference_vs30_type=measured
reference_vs30_value=800.0
risk_investigation_time=None

In [53]:
o = readinput.get_oqparam('job_hc_from_csv.ini')
readinput.get_sitecol_hcurves(o)

(<SiteCollection with 3 sites>,
 {'PGA': array([[ 0.4891846 ,  0.45525296,  0.39313824],
         [ 0.30783781,  0.24033782,  0.13052459],
         [ 0.9977148 ,  0.87679151,  0.20852236]]),
  'SA(0.2)': array([[ 0.414227  ,  0.19310681,  0.16763226],
         [ 0.17196265,  0.05421363,  0.02879748],
         [ 0.81992743,  0.61794658,  0.55046728]]),
  'SA(0.5)': array([[ 0.0414227 ,  0.01931068,  0.01676323],
         [ 0.0171965 ,  0.00542136,  0.00287974],
         [ 0.8199274 ,  0.61794658,  0.55046728]])})

# Writing a HelloWorld calculator

In [54]:
from openquake.calculators import base

In [55]:
HelloCalculator = base.calculators['hello']; HelloCalculator

my_calculators.hello.HelloCalculator

In [56]:
print open('hello.ini').read()
oqp = readinput.get_oqparam('hello.ini')

[general]
calculation_mode = hello
maximum_distance = 200
intensity_measure_types = PGA
export_dir = /tmp



In [57]:
from openquake.baselib.performance import PerformanceMonitor
calc = HelloCalculator(oqp, PerformanceMonitor('hello'))

In [58]:
calc.run()

ERROR:root:Cleanup error
Traceback (most recent call last):
  File "/home/michele/oq-risklib/openquake/calculators/base.py", line 135, in run
    self.clean_up()
  File "/home/michele/oq-risklib/openquake/calculators/base.py", line 200, in clean_up
    [(r.uid, r.weight) for r in self.rlzs_assoc.realizations], rlz_dt)
  File "/home/michele/oq-risklib/openquake/commonlib/datastore.py", line 400, in getter
    value = self.datastore[key]
  File "/home/michele/oq-risklib/openquake/commonlib/datastore.py", line 305, in __getitem__
    raise KeyError('No %r found in %s' % (key, self))
KeyError: "No 'rlzs_assoc' found in <DataStore 252>"


{}

In the export_dir there will be a file `performance.csv` with some interesting information about time spent and memory allocated

## Testing the calculator

In [228]:
from hello_test import HelloTestCase
HelloTestCase??

===================================================================================================================================

#     Parallel execution with apply_reduce

Example: building a frequency hystogram

In [59]:
from collections import Counter

In [60]:
Counter('pippolippo')

Counter({'i': 2, 'l': 1, 'o': 2, 'p': 5})

In [61]:
def frequencyhisto(fnames):
    "Compute the character frequency hystogram of a set of files"
    c = Counter()
    for fname in fnames:
        c += Counter(open(fname).read().upper())
    return c

In [62]:
from openquake.commonlib import nrml_examples

In [63]:
DIR = nrml_examples.__path__[0]

In [64]:
DIR

'/home/michele/oq-risklib/openquake/commonlib/nrml_examples'

In [65]:
import os; xmlfiles = [os.path.join(DIR, f) for f in os.listdir(DIR) if f.endswith('.xml')]; xmlfiles

['/home/michele/oq-risklib/openquake/commonlib/nrml_examples/hazard-map.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/hazard-map-mean.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/fragm_c.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/ses.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/hazard-curves-mean.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/uhs-mean.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/uhs.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/logic-tree-gmpe.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/exposure-portfolio.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/uhs-quantile.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/bcr-map.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml_examples/loss-map-quantile.xml',
 '/home/michele/oq-risklib/openquake/commonlib/nrml

In [66]:
len(xmlfiles)

37

In [67]:
apply(frequencyhisto, (xmlfiles,))

Counter({'\n': 1154,
         ' ': 12304,
         '!': 40,
         '"': 1726,
         "'": 92,
         '(': 6,
         ')': 6,
         '+': 41,
         ',': 53,
         '-': 363,
         '.': 1049,
         '/': 861,
         '0': 1383,
         '1': 586,
         '2': 405,
         '3': 275,
         '4': 257,
         '5': 246,
         '6': 167,
         '7': 167,
         '8': 177,
         '9': 181,
         ':': 286,
         '<': 1052,
         '=': 940,
         '>': 1052,
         '?': 72,
         'A': 1339,
         'B': 269,
         'C': 699,
         'D': 761,
         'E': 2177,
         'F': 312,
         'G': 781,
         'H': 353,
         'I': 1300,
         'K': 52,
         'L': 1389,
         'M': 1102,
         'N': 1410,
         'O': 1368,
         'P': 747,
         'Q': 53,
         'R': 1159,
         'S': 1545,
         'T': 1799,
         'U': 559,
         'V': 329,
         'W': 147,
         'X': 219,
         'Y': 211,
         'Z': 70,
     

In [68]:
from openquake.commonlib.parallel import apply_reduce
import operator

In [ ]:
apply_reduce(frequencyhisto, (xmlfiles,), agg=operator.add, acc=Counter(), concurrent_tasks=4)

In [ ]:
[len(chunk) for chunk in apply_reduce._chunks]

In [ ]:
apply_reduce(frequencyhisto, (xmlfiles,), agg=operator.add, acc=Counter(), concurrent_tasks=4,
             weight=os.path.getsize)

In [ ]:
[chunk.weight for chunk in apply_reduce._chunks]

In [243]:
[len(chunk) for chunk in apply_reduce._chunks]

[7, 10, 7, 8, 5]

In [244]:
apply_reduce._chunks[0]

<WeightedSequence ['/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/logic-tree-gmpe.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/logic-tree-source-model.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/complex-fault-rupture.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/loss-fractions-taxonomies.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/hazard-curves-mean.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/uhs-quantile.xml', '/home/ubuntu/oq-risklib/openquake/commonlib/nrml_examples/collapse-map.xml'], weight=8742>

In [245]:
from openquake.baselib.general import split_in_blocks

In [246]:
split_in_blocks??

# Accumulating dictionaries

In [247]:
from openquake.baselib.general import AccumDict
AccumDict??

In [248]:
acc = AccumDict()
acc

{}

In [249]:
acc2 = acc + {'a': 1}

In [250]:
acc3 = acc2 + {'a': 1}

In [251]:
acc3

{'a': 2}

In [252]:
acc4 = acc3 + {'b': 0}
acc4

{'a': 2, 'b': 0}

# Sequential scenario hazard calculator

In [253]:
from my_calculators import scenario
scenario??

# Parallel scenario hazard calculator

In [254]:
from openquake.calculators.hazard import ScenarioCalculator

In [255]:
ScenarioCalculator??

# Caveats and notes

1. is the parallel calculator faster than the sequential one?
2. is the export faster than the calculation?
3. how much the results depend on the seed?
4. is the memory occupation an issue?
5. do we really need a scenario hazard calculator?

In [256]:
from openquake.baselib.performance import PerformanceMonitor
PerformanceMonitor?

# Towards a Scenario Damage Calculator

In [257]:
from openquake.commonlib import readinput
from openquake.calculators.calc import calc_gmfs

In [258]:
o = readinput.get_oqparam('job_damage.ini')
exposure = readinput.get_exposure(o)
sitecol, assets_by_site = readinput.get_sitecol_assets(o, exposure)
risk_model = readinput.get_risk_model(o)
gmfs_by_imt = calc_gmfs(o, sitecol)

In [259]:
ri = risk_model.build_input('PGA', gmfs_by_imt['PGA'], assets_by_site)
ri

<RiskInput IMT=PGA, taxonomy=RM, weight=1>

In [260]:
assets, hazards, epsilons = ri.get_all()
print assets
print hazards
print epsilons

{'RM': [<Asset a1>]}
{'RM': [array([ 0.06108985,  0.02149156,  0.09326082,  0.017891  ,  0.06110326])]}
{'RM': [None]}


In [261]:
from openquake.risklib.riskinput import RiskInput; RiskInput

openquake.risklib.riskinput.RiskInput